#<b><center>NLP Pennsylvania</b></center>

Importacion de librerías

In [1]:
from google.colab import drive
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from google.cloud import storage
from google.colab import auth
auth.authenticate_user()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Montando carpeta drive.

In [2]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive/datalake/reviews-estados/review-Pennsylvania

Mounted at /content/drive
/content/drive/MyDrive/datalake/reviews-estados/review-Pennsylvania


In [3]:
df = pd.read_parquet('reviews.parquet')
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1318329 entries, 0 to 1318328
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   user_id  1318329 non-null  float64
 1   gmap_id  1318329 non-null  object 
 2   text     1318329 non-null  object 
dtypes: float64(1), object(2)
memory usage: 30.2+ MB


,user_id,gmap_id,text
0,1.170220e+20,0x89c8c1a20671e5d5:0x1ca654f3761c121a,"Very knowledgeable, very helpful. Great place..."
1,1.147554e+20,0x89c8c1a20671e5d5:0x1ca654f3761c121a,One of the best selections of vape juice and t...
2,1.014546e+20,0x89c8c1a20671e5d5:0x1ca654f3761c121a,Great and friendly place. Interesting discoun...
3,1.111469e+20,0x89c8c1a20671e5d5:0x1ca654f3761c121a,Friendliest staff and a huge selection of vape...
4,1.027780e+20,0x89c8c1a20671e5d5:0x1ca654f3761c121a,"My new go to for vape supplies, the people kno..."


In [4]:
textos = df['text']
textos.tail()

1318324    Great staff, excellent selection of brews. Hop...
1318325    Safe and wholesome neighborhood bar and grill ...
1318326    Great place to hangout with friends or meet ne...
1318327    Recently had lunch here during happy hour (5-7...
1318328    Fantastic staff, friendly and welcoming. Great...
Name: text, dtype: object

<b>Tokenización</b>

Se eliminan signos de puntuación y se toma la raíz de las palabras. Se tokeniza el texto de los reviews.

In [5]:
# Initialize empty array
# to append clean text
corpus = []
for texto in textos:

    # column : "Review", row ith
    review = re.sub('[^a-zA-Z]', ' ', texto)

    # convert all cases to lower cases
    review = review.lower()

    # split to array(default delimiter is " ")
    review = review.split()

    # creating PorterStemmer object to
    # take main stem of each word
    ps = PorterStemmer()

    # loop for stemming each word
    # in string array at ith row
    review = [ps.stem(word) for word in review
                if not word in set(stopwords.words('english'))]

    # rejoin all string array elements
    # to create back into a string
    review = ' '.join(review)

    # append each string to create
    # array of clean text
    corpus.append(review)



In [6]:
df['review'] = corpus
df.drop(columns=['text'])
df.info()
df.to_parquet('reviews_token.parquet')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1318329 entries, 0 to 1318328
Data columns (total 4 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   user_id  1318329 non-null  float64
 1   gmap_id  1318329 non-null  object 
 2   text     1318329 non-null  object 
 3   review   1318329 non-null  object 
dtypes: float64(1), object(3)
memory usage: 40.2+ MB


Guardado en datalake de  Google Cloud:

In [7]:
project_id = 'deep-wave-407518'
bucket_name = 'yelp-and-maps-data-processed'


In [8]:
# Google Storage user.parquet
# Configura el cliente de almacenamiento
client = storage.Client(project=project_id)

# Obtén el bucket
bucket = client.get_bucket(bucket_name)

# Define la ruta del archivo Parquet en Google Cloud Storage
gcs_path = 'Maps/reviews-estados/review-Pennsylvania/reviews_token.parquet'.format(bucket_name)

# Sube el archivo Parquet a Google Cloud Storage
blob = bucket.blob(gcs_path)
blob.upload_from_filename('reviews_token.parquet')
